In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
Titles = pd.read_csv('Titles_English.csv')

In [3]:
Titles = Titles.drop('Unnamed: 0', axis=1)
Titles['EAN'] = Titles['EAN'].astype(str)

In [4]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1000, ngram_range=(1,1), stop_words='english')
features = tfidf.fit_transform(Titles['Annotation Text']).toarray()
labels = Titles.Category
features.shape

(300359, 3272)

In [5]:
Titles_train, Titles_test = train_test_split(Titles, random_state=42)

In [6]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MaxAbsScaler

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=42)

# Initialize the MaxAbsScaler
scaler = MaxAbsScaler()

# Fit on the training data and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Just transform the test data
X_test_scaled = scaler.transform(X_test)

# Train the Logistic Regression model on the scaled data
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train_scaled, y_train)

# Get the predicted labels and probabilities on the scaled test data
y_pred = logreg.predict(X_test_scaled)
y_pred_proba = logreg.predict_proba(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy*100:.2f}%')

# Create a dataframe with the predicted labels and maximum probabilities (confidence scores)
results_df = pd.DataFrame({
    'Predicted Category': y_pred,
    'Confidence Score': np.max(y_pred_proba, axis=1)
}, index=Titles_test.index)  # use the same indices as in Titles_test

# Add the 'Annotation Text' column to the results
results_df = Titles_test[['Annotation Text']].join(results_df)

# Print the results
print(results_df)

C:\Users\Pupnsuds90\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model accuracy: 71.16%
                                          Annotation Text  Predicted Category  \
272607  'You don't want to mind about any of this, ' s...                   1   
40303   "General Grant is emphatically a man, not of w...                   1   
172631  This work has been selected by scholars as bei...                   0   
286961  They say one never forgets his or her first lo...                   0   
191935  This work has been selected by scholars as bei...                   0   
...                                                   ...                 ...   
182491  Very Vanilla Bean ice cream doesn't feel very ...                   0   
23941   'Who am I? What are my roots?' These are quest...                   1   
2172    Every Kid's Guide to Coping With Childhood Tra...                   0   
114632  Victoria Dahl is a self-made multi-billionaire...                   0   
153901  Composed for the Norfolk and Norwich Festival,...                   1   

    

In [8]:
results_df = Titles_test[['Title']].join(results_df)
print(results_df)

                                                    Title  \
272607  Dancing Bear                                  ...   
40303   Life of General Ulysses S. Grant: Containing a...   
172631  Adam's Illustrated Guide to Rye (with map): Wi...   
286961  Angela's Letter                               ...   
191935  Memoirs of a Professional Lady Nurse          ...   
...                                                   ...   
182491  Feeling Special                               ...   
23941   My Father's People                            ...   
2172    Every Kid's Guide to Coping with Childhood Tra...   
114632  Nevermore...                                  ...   
153901  Sea Pictures, Op.37: Vocal score (Urtext)     ...   

                                          Annotation Text  Predicted Category  \
272607  'You don't want to mind about any of this, ' s...                   1   
40303   "General Grant is emphatically a man, not of w...                   1   
172631  This work has be

In [11]:
results_df[results_df['Predicted Category'] == 1].sort_values('Confidence Score', ascending = False)

,Title,Annotation Text,Predicted Category,Confidence Score
112644,Boosey's Guide to the Opera - Containing the P...,"First published in 1880, ""Boosey's Guide to th...",1,0.999871
113136,Jeweled Bearings for Watches - A Full and Comp...,This vintage book contains a detailed treatise...,1,0.999863
44145,Select Treatise of St. Athanasius in Controver...,This book contains the first volume of John He...,1,0.999860
104100,Antiques of Limerick and its Neighbourhood ...,This vintage book contains a detailed guide to...,1,0.999858
43496,Badminton Library of Sports and Pastimes - Cou...,This vintage volume contains a comprehensive g...,1,0.999847
...,...,...,...,...
9813,Living Tree ...,The Living Tree gives a child-like explanation...,1,0.500026
232572,Romantic and healthy relationship: Make love a...,So many couples suffer in their sexual life be...,1,0.500009
66637,Ezekiel's Wheel Within A Wheel ...,There is no other Prophet who has stood outsid...,1,0.500007
107483,Return to the Sea ...,John Egan is a Sydney poet who also lives on t...,1,0.500005
